<a href="https://colab.research.google.com/github/Senremi7/Climate-change-analysis-granite/blob/main/climate_change_analysis_granite_Jeremiah.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install replicate
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:
from google.colab import userdata
import os

# ambil api_token dari secret
api_token = userdata.get("api_token")

# lempar api_token ke env. var.
os.environ["REPLICATE_API_TOKEN"] = api_token

In [ ]:
from langchain_community.llms import Replicate

# Get the llm
llm = Replicate(model="ibm-granite/granite-3.3-8b-instruct")

In [ ]:
pip install kagglehub[pandas-datasets]

In [ ]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

In [ ]:
file_path = "global_warming_dataset.csv"

In [ ]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "ankushpanday1/global-warming-dataset-195-countries-1900-2023",
  file_path,
  # Provide any additional arguments like
  # sql_query or pandas_kwargs. See the
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

print("First 5 records:", df.head())

In [ ]:
df

In [ ]:
df["Country"].unique()


In [ ]:
prompt = f"""
You are a data cleaning AI assistant.
I have a dataset of global warming data from 1900 to 2023 for 195 countries (file: global_warming_dataset.csv).

Your task:
1. Check for missing or corrupted data.
2. Provide step-by-step instructions to clean the dataset.

Respond clearly and concisely with cleaning steps only.
"""
output = llm.invoke(prompt)
print(output)


In [ ]:
df.isnull().sum()

In [ ]:
!pip install langchain_experimental

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from langchain_experimental.agents.agent_toolkits.pandas.base import create_pandas_dataframe_agent

agent = create_pandas_dataframe_agent(
    llm=llm,
    df=df,
    verbose=True,
    allow_dangerous_code=True
)

# Prompt untuk analisis dataset
prompt = "Berikan analisis ringkas tentang dataset ini dan buatkan plot visualnya terkait tren perubahan iklim."

# Granite untuk analisis
output = agent.invoke({"input": prompt})

# Tampilkan hasil
print("Hasil Analisis:\n", output)

result = agent.invoke(
    "Tampilkan 5 baris pertama dari df_climate dalam format tabel Pandas, bukan markdown atau text."
)
print(result['output'])

Karena Granite kurang bagus dalam membuat grafik maka analisis chart dilakukan manual, Granite akan melakukan analisisnaratif saja, seperti pola dan insight

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
# year vs Temperature_Anomaly
import seaborn as sns
import plotly.express as px
plt.figure(figsize=(10,8))
sns.lineplot(data=df, x='Year', y='Temperature_Anomaly', color='crimson', linewidth=2)

plt.title("Yearly Trend of Temperature Anomaly", fontsize=14, fontweight="bold")
plt.xlabel("Year")
plt.ylabel("Temperature Anomaly")
plt.grid(True)
plt.legend(["Temperature Anomaly"], loc="best")

plt.show()

In [ ]:
# Calculate the correlation matrix
correlation_matrix = df.corr(numeric_only=True)

# Plot the heatmap
plt.figure(figsize=(15, 12))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix Heatmap')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(
    x='Policy_Score',
    y='Renewable_Energy_Usage',
    size='CO2_Emissions',
    hue='Air_Pollution_Index',
    data=df,
    sizes=(20, 400),
    palette='coolwarm',
    alpha=0.6,
    edgecolor='black'
)

plt.title('Impact of Policy on Renewable Energy Adoption', fontsize=14, fontweight='bold')
plt.xlabel('Policy Effectiveness Score', fontsize=12)
plt.ylabel('Renewable Energy Usage (%)', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.5)
plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
plt.tight_layout()
plt.show()

In [ ]:
df['Decade'] = ((df['Year'] // 10) * 10).astype(str) + 's'

In [ ]:
df_gl_aml = df.groupby('Decade')['Temperature_Anomaly'].mean().reset_index()

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(df_gl_aml['Decade'], df_gl_aml['Temperature_Anomaly'], marker='o', linestyle='-', color='b')

#customize the plot
plt.title('Temperature Anamoly by Decade')
plt.xlabel('Decade')
plt.ylabel('Temperature Anomaly')
plt.xticks(rotation=45)
plt.grid(True)

#Display the plot
plt.tight_layout()
plt.show()

In [ ]:
time_series_var = 'CO2_Emissions'
plt.figure(figsize=(12, 6))
sns.lineplot(df, x='Year', y=time_series_var)
plt.title(f"Trend of {time_series_var} Over Time")
plt.xlabel("Year")
plt.ylabel(time_series_var)
plt.grid(True)
plt.show()

In [ ]:
time_series_var = 'Deforestation_Rate'
plt.figure(figsize=(12, 6))
sns.lineplot(df, x='Year', y=time_series_var)
plt.title(f"Trend of {time_series_var} Over Time")
plt.xlabel("Year")
plt.ylabel(time_series_var)
plt.grid(True)
plt.show()

In [ ]:
time_series_var = 'Average_Rainfall'
plt.figure(figsize=(12, 6))
sns.lineplot(df, x='Year', y=time_series_var)
plt.title(f"Trend of {time_series_var} Over Time")
plt.xlabel("Year")
plt.ylabel(time_series_var)
plt.grid(True)
plt.show()

In [ ]:
agent.invoke({"input": """Analisis kolom 'Temperature_Anomaly'. Jelaskan tren keseluruhan dari tahun ke tahun, sebutkan periode perubahan signifikan, serta apa implikasinya terhadap pola perubahan iklim global. Jawaban harus jelas, ringkas, tidak berputar-putar, gunakan bahasa sederhana, berikan poin utama, lalu simpulkan secara singkat.
"""}, handle_parsing_errors=True)

In [ ]:
prompt = """Analisis kolom 'CO2_Emissions'.
Bagaimana perkembangan emisi dari waktu ke waktu, apakah ada kaitannya dengan populasi atau aktivitas industri,
dan apa wawasan yang bisa diambil untuk kebijakan iklim.
Jawaban harus jelas, ringkas, tidak berputar-putar, gunakan bahasa sederhana, berikan poin utama, lalu simpulkan secara singkat.
"""

output = llm.invoke(prompt)
print("Output Granite:\n", output)


In [ ]:
prompt = f"""
Analisis kolom 'Sea_Level_Rise'. Jelaskan tren jangka panjang, apakah ada percepatan dalam beberapa dekade terakhir, dan hubungkan hasil ini dengan indikator iklim lain jika relevan.

Jawaban harus:
- Jelas dan ringkas
- Tidak berputar-putar
- Gunakan bahasa sederhana
- Berikan poin utama
- Akhiri dengan kesimpulan singkat
"""

print("Prompt yang dikirim ke Granite:\n", prompt)

# Panggil Granite langsung
output = llm.invoke(prompt)

print("Output Granite:\n", output)


In [ ]:
prompt = f"""
Analisis hubungan antara 'Forest_Area' dan 'Deforestation_Rate'.
Apakah luas hutan menurun, seberapa cepat deforestasi terjadi, dan apa dampaknya terhadap ekosistem.

Jawaban harus:
- Jelas dan ringkas
- Tidak berputar-putar
- Gunakan bahasa sederhana
- Berikan poin utama
- Akhiri dengan kesimpulan singkat
"""

print("Prompt yang dikirim ke Granite:\n", prompt)

# Panggil Granite langsung
output = llm.invoke(prompt)

print("Output Granite:\n", output)


In [ ]:
prompt = f"""
Analisis kolom 'Renewable_Energy_Usage'. Apakah penggunaannya meningkat dari waktu ke waktu, bagaimana perbandingannya dengan penggunaan bahan bakar fosil, serta apa artinya bagi transisi energi.

Jawaban harus:
- Jelas dan ringkas
- Tidak berputar-putar
- Gunakan bahasa sederhana
- Berikan poin utama
- Akhiri dengan kesimpulan singkat
"""

print("Prompt yang dikirim ke Granite:\n", prompt)

# Panggil Granite langsung
output = llm.invoke(prompt)

print("Output Granite:\n", output)


In [ ]:
prompt = f"""
Analisis kolom 'Biodiversity_Index'.
Apakah keanekaragaman hayati mengalami penurunan, apakah ada hubungannya dengan deforestasi atau polusi,
serta apa pentingnya bagi keberlanjutan lingkungan.

Jawaban harus:
- Jelas dan ringkas
- Tidak berputar-putar
- Gunakan bahasa sederhana
- Berikan poin utama
- Akhiri dengan kesimpulan singkat
"""

print("Prompt yang dikirim ke Granite:\n", prompt)

# Panggil Granite langsung
output = llm.invoke(prompt)

print("Output Granite:\n", output)


In [ ]:
prompt = f"""
Analisis kolom 'Air_Pollution_Index'.
Apakah indeks polusi udara cenderung meningkat atau menurun, faktor apa yang memengaruhinya, dan bagaimana dampaknya terhadap kesehatan masyarakat.

Jawaban harus:
- Jelas dan ringkas
- Tidak berputar-putar
- Gunakan bahasa sederhana
- Berikan poin utama
- Akhiri dengan kesimpulan singkat
"""

print("Prompt yang dikirim ke Granite:\n", prompt)

# Panggil Granite langsung
output = llm.invoke(prompt)

print("Output Granite:\n", output)
